In [10]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [13]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=4*28, step_size=1)

2025-05-11 08:03:01,726 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:03:01,731 INFO: Initializing external client
2025-05-11 08:03:01,731 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 08:03:02,219 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Fetching data from 2025-04-12 12:03:01.726256+00:00 to 2025-05-11 11:03:01.726256+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 


In [19]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 08:04:48,085 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:04:48,085 INFO: Initializing external client
2025-05-11 08:04:48,085 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 08:04:48,707 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [20]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [21]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,6072,117.0,2025-05-11 13:00:00+00:00
1,6072,58.0,2025-05-11 13:00:00+00:00
2,5626,104.0,2025-05-11 13:00:00+00:00
3,5626,156.0,2025-05-11 13:00:00+00:00
4,5779,139.0,2025-05-11 13:00:00+00:00
5,5779,163.0,2025-05-11 13:00:00+00:00
6,5788,126.0,2025-05-11 13:00:00+00:00
7,5788,103.0,2025-05-11 13:00:00+00:00
8,5905,118.0,2025-05-11 13:00:00+00:00
9,5905,101.0,2025-05-11 13:00:00+00:00


In [22]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-05-11 08:04:56,428 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:04:56,428 INFO: Initializing external client
2025-05-11 08:04:56,428 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 08:04:56,973 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [23]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/thanoojl/Resources/jobs/citi_bike_prediction_1_offline_fg_materialization/config_1746963961028) to trigger the materialization job again.


(Job('citi_bike_prediction_1_offline_fg_materialization', 'SPARK'), None)